In [1]:
import os
import pandas as pd
import gff3

In [40]:
### change crac_file and HTF_gene according to HTF_gene
crac_file = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_19thJune2024/ratios/Jag_ratios.txt'
HTF_gene = 'jag'

crac_table = pd.read_csv(crac_file, sep = '\t', usecols = ['Gene', f'RPKMPW_{HTF_gene}', f'{HTF_gene}_ratio'])

gtf_file = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/Annotation_files/B_subtilis_CRAC_RNAseq_analysis.gtf'
gtf = pd.read_csv(gtf_file, sep = '\t', names = gff3.header)
gtf['name'] = [i.split(' ')[1] for i in gtf['attributes'].tolist()]

converter_file = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/Gene_categories/Subti_Genbank_GO_conversion_table.txt'
converter = pd.read_csv(converter_file, sep = '\t', header = 0)

genbank_file = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/Annotation_files/B_subtilis_GenBank_w_name_and_locus.gff3'
genbank = pd.read_csv(genbank_file, sep = '\t', header = 0)
genbank = genbank.drop_duplicates(['name'])

RNAseq_file = f'/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/DEG/{HTF_gene}_DEG.txt'
RNAseq = pd.read_csv(RNAseq_file, sep = '\t', header = 0)
RNAseq.fillna('na', inplace=True)

output_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_19thJune2024/ratios/thesis_tables'

In [41]:
### We will make a table with the following columns
### pyRAP_gene
### feature
### Gene_base
### locus
### Associated_GO_terms
### RPKMPW
### crac_ratio
### log2foldchange
### padj
pyRAP_gene = crac_table['Gene']
feature = []
Gene_base = []
locus = []
GO_terms = []
RPKMPW = crac_table[f'RPKMPW_{HTF_gene}']
# crac_ratio = crac_table[f'{HTF_gene}_ratio']
log2foldchange = []
padj = []
for gene in pyRAP_gene:
    feature.append(gtf['source'][gtf['name'] == gene].iloc[0])
    Gene_base.append('.'.join(gene.split('.')[:-1]))
    
    gene = gene.replace('YP_', 'YP*').replace('NP_', 'NP*')
    gene_first = '.'.join(gene.split('.')[:-1]).split('_')[0]
    gene_first = gene_first.replace('*', '_')
    try:
        locus.append(converter['locus'][converter['Subti_name'] == gene_first].iloc[0])
    except:
        try:
            locus.append(converter['locus'][converter['GenBank_name'] == gene_first].iloc[0])
        except:
            try:
                locus.append(genbank['locusTag'][genbank['name'] == gene_first].iloc[0])
            except:
                print(gene)
                print(gene_first)
                break

for gene in Gene_base:
    try:
        log2foldchange.append(RNAseq['log2FoldChange'][RNAseq['Gene_base'] == gene].iloc[0])
    except:
        log2foldchange.append('na')
    try:
        padj.append(RNAseq['padj'][RNAseq['Gene_base'] == gene].iloc[0])
    except:
        padj.append('na')

for l in locus:
    GO_terms.append(converter['Gene_categories'][converter['locus'] == l].iloc[0])

thesis_table = pd.DataFrame()
thesis_table['pyRAP_Gene'] = pyRAP_gene
thesis_table['feature'] = feature
thesis_table['Gene_base'] = Gene_base
thesis_table['locus'] = locus
thesis_table['GO_terms'] = GO_terms
thesis_table['RPKMPW'] = RPKMPW
# thesis_table['crac_ratio'] = crac_ratio
thesis_table['log2foldchange'] = log2foldchange
thesis_table['padj'] = padj

In [42]:
thesis_table[:10]

,pyRAP_Gene,feature,Gene_base,locus,GO_terms,RPKMPW,log2foldchange,padj
0,bmrB.1,putative_sRNA,bmrB,BSU_09700,Other regulators,50.826271,0.391389,na
1,YP_009514003.1.2,CDS,YP_009514003.1,BSU_34399,na,50.547085,na,na
2,walM.5,3UTR,walM,BSU_06250,"Cell wall/ other, Secreted proteins",27.385281,na,na
3,walM_ydjN.1,intergenic_UTR,walM_ydjN,BSU_06250,"Cell wall/ other, Secreted proteins",23.881356,na,na
4,metQ.1,3UTR,metQ,BSU_32730,Biosynthesis/ acquisition of methionine/ S-ade...,5.394432,0.237246,0.722921
5,mnrW_ykoY.2,transcript,mnrW_ykoY,BSU_misc_RNA_87,na,4.930709,na,na
6,ypoP.3,3UTR,ypoP,BSU_21700,Transcription factor/ other/ based on similarity,4.095542,0.079623,0.93204
7,ykoX.5,3UTR,ykoX,BSU_13430,"Membrane proteins, Other transporters",4.093717,0.152715,0.946816
8,mswD.1,transcript,mswD,BSU_misc_RNA_21,na,4.037768,na,na
9,ykoY.1,5UTR,ykoY,BSU_13440,Newly identified sporulation proteins (based o...,3.614690,na,na


In [34]:
thesis_table[:10]

,pyRAP_Gene,feature,Gene_base,locus,GO_terms,RPKMPW,log2foldchange,padj
0,bmrB.1,putative_sRNA,bmrB,BSU_09700,Other regulators,50.826271,0.391389,NaN
1,YP_009514003.1.2,CDS,YP_009514003.1,BSU_34399,na,50.547085,na,na
2,walM.5,3UTR,walM,BSU_06250,"Cell wall/ other, Secreted proteins",27.385281,na,na
3,walM_ydjN.1,intergenic_UTR,walM_ydjN,BSU_06250,"Cell wall/ other, Secreted proteins",23.881356,na,na
4,metQ.1,3UTR,metQ,BSU_32730,Biosynthesis/ acquisition of methionine/ S-ade...,5.394432,na,na
5,mnrW_ykoY.2,transcript,mnrW_ykoY,BSU_misc_RNA_87,na,4.930709,na,na
6,ypoP.3,3UTR,ypoP,BSU_21700,Transcription factor/ other/ based on similarity,4.095542,na,na
7,ykoX.5,3UTR,ykoX,BSU_13430,"Membrane proteins, Other transporters",4.093717,na,na
8,mswD.1,transcript,mswD,BSU_misc_RNA_21,na,4.037768,na,na
9,ykoY.1,5UTR,ykoY,BSU_13440,Newly identified sporulation proteins (based o...,3.614690,na,na


In [8]:
astring = 'YP_009514003.1.2'
astring.replace('YP_', 'YP*')

'YP*009514003.1.2'

In [37]:
print(len(RNAseq))
print(len(RNAseq.drop_duplicates(['Gene_base'])))

6637
6637
